# 1. Bike

## 1.0 Startup

In [1]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics

executing gpml startup script...


In [2]:
dataset = 'bike_data'
load ('./data/bike_data.mat');
[ndata, ~] = size(M);
X = M(:,1:14);
Y = M(:,15);
R = randperm(ndata);        
xt = X(R(1:2379),:);
yt = Y(R(1:2379));
R(1:2379) = [];
x = X(R,:);          
y = Y(R);
n = 15000; nt = 2379;   %Ms = 15

dataset = bike_data


In [3]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'N' ; opts.Ynorm = 'N' ;
opts.Ms = 30 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;

partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;
save('./res/bike30', 'models','t_dGP_train')

In [4]:
partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
s = load('./res/bike30');
models = s.models;


In [52]:
criterions = {'BCM','PoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing bike_data kmeans BCM
BCM: SMSE 0.00068380, MSLL -12.9848, NLPD -6.3713
Processing bike_data kmeans PoE
PoE: SMSE 0.00070672, MSLL -12.9847, NLPD -6.3713


## 1.1 RBCM vs TERBCM

### 1.1.1 RBCM baseline

In [53]:
criterions = {'RBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing bike_data kmeans RBCM
RBCM: SMSE 0.00048953, MSLL -4.0999, NLPD 2.5136


### 1.1.2 Grid q

In [54]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TERBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:2.0;

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
        for q = qs
            [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
            [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
            printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
            filename = sprintf('./res/terbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
            save(filename, 'SMSE','MSLL','NLPD')
        endfor
    endfor
endfor


Processing bike_data kmeans TERBCM
TERBCM: SMSE 0.00066865, MSLL 1358.8803, NLPD 1365.4938 - q   0.20
TERBCM: SMSE 0.00065218, MSLL 249.7480, NLPD 256.3614 - q   0.40
TERBCM: SMSE 0.00062059, MSLL 50.5733, NLPD 57.1867 - q   0.60
TERBCM: SMSE 0.00056664, MSLL 6.4122, NLPD 13.0256 - q   0.80
TERBCM: SMSE 0.00048953, MSLL -4.0999, NLPD 2.5136 - q   1.00
TERBCM: SMSE 0.00040219, MSLL -5.2599, NLPD 1.3535 - q   1.20
TERBCM: SMSE 0.00032431, MSLL -0.8359, NLPD 5.7776 - q   1.40
TERBCM: SMSE 0.00026684, MSLL 13.2497, NLPD 19.8632 - q   1.60
TERBCM: SMSE 0.00022965, MSLL 52.4271, NLPD 59.0406 - q   1.80
TERBCM: SMSE 0.00020808, MSLL 163.8566, NLPD 170.4700 - q   2.00


### 1.1.3 Find single q

In [73]:
criterion = 'TERBCM';

In [74]:
printf('Optimizing q\r\n');
q1 = grad_q(models, criterion, opts, iter=1000, lr=1e-9, lambda=1e6, init_q=1.2)
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q1) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

Optimizing q
Start preprocessing...
Finish preprocessing...
Optimizing q: remain iter - 1000, q - 1.2000
q1 =  1.3271
TERBCM: SMSE 0.00035061, MSLL -3.1951, NLPD 3.4183


### 1.1.4 Find multi q

In [75]:
criterion = 'TERBCM';

In [77]:
printf('Optimizing q\r\n');
iqs = 1.2 + 0.01 * rand(opts.Ms,1)
for ldk=-1:3
    ld = 10^ldk
    qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-3, lambda=ld, init_q=iqs)
    [mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
endfor

Optimizing q
iqs =

   1.2058
   1.2063
   1.2048
   1.2044
   1.2018
   1.2086
   1.2044
   1.2039
   1.2045
   1.2073
   1.2076
   1.2040
   1.2096
   1.2027
   1.2012
   1.2032
   1.2004
   1.2054
   1.2099
   1.2007
   1.2035
   1.2063
   1.2071
   1.2014
   1.2041
   1.2018
   1.2045
   1.2056
   1.2084
   1.2080

ld =  0.10000
Start preprocessing...
k =  1
k =  2
k =  3
k =  4
k =  5
k =  6
k =  7
k =  8
k =  9
k =  10
k =  11
k =  12
k =  13
k =  14
k =  15
k =  16
k =  17
k =  18
k =  19
k =  20
k =  21
k =  22
k =  23
k =  24
k =  25
k =  26
k =  27
k =  28
k =  29
k =  30
Finish preprocessing...
qs =

   0.53444
   0.59274
   0.57247
   0.55216
   0.64296
   0.65472
   0.56372
   0.56042
   0.60058
   0.63884
   0.59535
   0.56536
   0.63289
   0.54681
   0.57837
   0.54656
   0.55630
   0.57632
   0.58821
   0.55353
   0.57202
   0.63899
   0.56564
   0.58227
   0.56947
   0.55606
   0.59314
   0.63959
   0.57860
   0.59140

TERBCM: SMSE 0.00072170, MSLL 66.0300, NLPD 72.643

In [79]:
printf('Optimizing q\r\n');
iqs = 2.0 + 0.01 * rand(opts.Ms,1)
for ldk=-1:3
    ld = 10^ldk
    qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-5, lambda=ld, init_q=iqs)
    [mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
endfor

Optimizing q
iqs =

   2.0086
   2.0069
   2.0037
   2.0085
   2.0071
   2.0020
   2.0067
   2.0012
   2.0044
   2.0086
   2.0087
   2.0024
   2.0005
   2.0082
   2.0010
   2.0005
   2.0096
   2.0031
   2.0045
   2.0091
   2.0099
   2.0001
   2.0033
   2.0080
   2.0027
   2.0068
   2.0038
   2.0023
   2.0007
   2.0025

ld =  0.10000
Start preprocessing...
k =  1
k =  2
k =  3
k =  4
k =  5
k =  6
k =  7
k =  8
k =  9
k =  10
k =  11
k =  12
k =  13
k =  14
k =  15
k =  16
k =  17
k =  18
k =  19
k =  20
k =  21
k =  22
k =  23
k =  24
k =  25
k =  26
k =  27
k =  28
k =  29
k =  30
Finish preprocessing...
qs =

   2.4133
   2.1490
   2.0281
   2.0794
   2.0123
   2.0036
   2.0716
   2.1982
   2.0788
   2.0142
   2.0368
   2.0435
   2.0099
   2.1668
   2.1269
   2.1008
   2.0889
   2.1133
   2.0224
   2.1983
   2.1954
   2.0051
   2.1981
   2.1521
   2.0324
   2.1339
   2.0649
   2.0132
   2.1941
   2.0198

TERBCM: SMSE 0.00008366, MSLL 717.5739, NLPD 724.1873
ld =  1
Start preprocessin

## 1.2 GRBCM vs TEGRBCM

### 1.2.1 GRBCM baseline

In [80]:
criterions = {'TEGRBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion, q=1) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing bike_data kmeans TEGRBCM
TEGRBCM: SMSE 0.00000352, MSLL -13.6828, NLPD -7.0694


### 1.2.2 Grid q

In [81]:
criterions = {'TEGRBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:4.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegrbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing bike_data kmeans TEGRBCM
TEGRBCM: SMSE 0.00000357, MSLL -13.2906, NLPD -6.6771 - q   0.20
TEGRBCM: SMSE 0.00000353, MSLL -13.3481, NLPD -6.7346 - q   0.40
TEGRBCM: SMSE 0.00000350, MSLL -13.4503, NLPD -6.8368 - q   0.60
TEGRBCM: SMSE 0.00000350, MSLL -13.5936, NLPD -6.9801 - q   0.80
TEGRBCM: SMSE 0.00000352, MSLL -13.6828, NLPD -7.0694 - q   1.00
TEGRBCM: SMSE 0.00000356, MSLL -13.2619, NLPD -6.6485 - q   1.20
TEGRBCM: SMSE 0.00000363, MSLL -10.6352, NLPD -4.0217 - q   1.40
TEGRBCM: SMSE 0.00000372, MSLL 0.2321, NLPD 6.8455 - q   1.60
TEGRBCM: SMSE 0.00000383, MSLL 41.1378, NLPD 47.7513 - q   1.80
TEGRBCM: SMSE 0.00000399, MSLL 192.2372, NLPD 198.8506 - q   2.00
TEGRBCM: SMSE 0.00000429, MSLL 752.0824, NLPD 758.6959 - q   2.20
TEGRBCM: SMSE 0.00000482, MSLL 2841.8583, NLPD 2848.4718 - q   2.40
TEGRBCM: SMSE 0.00000559, MSLL 10699.8268, NLPD 10706.4403 - q   2.60
TEGRBCM: SMSE 0.00000726, MSLL 40454.5432, NLPD 40461.1566 - q   2.80
TEGRBCM: SMSE 0.00000936, MSLL 153830.8161,

### 1.2.3 Find single q

In [87]:
criterion = 'TEGRBCM';

In [88]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=1000, lr=1e-9, lambda=1e6, init_q=1.001);

Optimizing q
Start preprocessing...
Finish preprocessing...
Optimizing q: remain iter - 1000, q - 1.0010


In [89]:
q

q =  0.98788


In [90]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.2.4 Find multi q

In [5]:
criterion = 'TEGRBCM';

In [8]:
printf('Optimizing q\r\n');
iqs = 1.0001 + 0.01 * rand(opts.Ms,1)
for ldk=-1:3
    ld = 10^ldk
    qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
    [mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
endfor

Optimizing q
iqs =

   1.0080
   1.0054
   1.0071
   1.0063
   1.0045
   1.0042
   1.0054
   1.0093
   1.0070
   1.0095
   1.0053
   1.0035
   1.0067
   1.0077
   1.0033
   1.0025
   1.0069
   1.0083
   1.0006
   1.0085
   1.0020
   1.0050
   1.0095
   1.0044
   1.0022
   1.0095
   1.0024
   1.0032
   1.0002
   1.0094

ld =  0.10000
Start preprocessing...
k =  1
k =  2
k =  3
k =  4
k =  5
k =  6
k =  7
k =  8
k =  9
k =  10
k =  11
k =  12
k =  13
k =  14
k =  15
k =  16
k =  17
k =  18
k =  19
k =  20
k =  21
k =  22
k =  23
k =  24
k =  25
k =  26
k =  27
k =  28
k =  29
k =  30
Finish preprocessing...
qs =

   1.0080
   1.0054
   1.0071
   1.0062
   1.0045
   1.0042
   1.0088
   1.0138
   1.0237
   1.0094
   1.0071
   1.0034
   1.0067
   1.0092
   1.0066
   1.0026
   1.0080
   1.0109
   1.0006
   1.0152
   1.0257
   1.0050
   1.0231
   1.0044
   1.0022
   1.0101
   1.0028
   1.0031
   1.0096
   1.0098

TEGRBCM: SMSE 0.00000009, MSLL -13.8204, NLPD -7.2216
ld =  1
Start preprocessin

In [7]:
printf('Optimizing q\r\n');
iqs = 0.8 + 0.01 * rand(opts.Ms,1)
for ldk=-1:3
    ld = 10^ldk
    qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-2, lambda=ld, init_q=iqs)
    [mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
endfor

Optimizing q
iqs =

   0.80105
   0.80871
   0.80467
   0.80164
   0.80651
   0.80445
   0.80082
   0.80778
   0.80588
   0.80602
   0.80653
   0.80858
   0.80821
   0.80043
   0.80722
   0.80447
   0.80367
   0.80542
   0.80872
   0.80888
   0.80208
   0.80086
   0.80813
   0.80642
   0.80370
   0.80857
   0.80744
   0.80216
   0.80976
   0.80252

ld =  0.10000
Start preprocessing...
k =  1
k =  2
k =  3
k =  4
k =  5
k =  6
k =  7
k =  8
k =  9
k =  10
k =  11
k =  12
k =  13
k =  14
k =  15
k =  16
k =  17
k =  18
k =  19
k =  20
k =  21
k =  22
k =  23
k =  24
k =  25
k =  26
k =  27
k =  28
k =  29
k =  30
Finish preprocessing...
qs =

   0.80105
   0.80871
   0.80464
   0.80127
   0.80652
   0.80445
   0.80238
   0.81006
   0.81549
   0.80602
   0.80668
   0.80831
   0.80827
   0.80109
   0.80876
   0.80415
   0.80366
   0.80644
   0.80870
   0.81178
   0.81434
   0.80092
   0.81540
   0.80608
   0.80367
   0.80844
   0.80722
   0.80184
   0.81571
   0.80216

TEGRBCM: SMSE 0.0000

## 1.3 GPoE vs TEGPoE

### 1.3.1 GPoE baseline

In [9]:
criterions = {'GPoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing bike_data kmeans GPoE
GPoE: SMSE 0.00000870, MSLL -7.6332, NLPD -1.0343


### 1.3.2 Grid q

In [10]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TEGPoE'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:4.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegpoe%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing bike_data kmeans TEGPoE
TEGPoE: SMSE 0.00001536, MSLL 942.2206, NLPD 948.8195 - q   0.20
TEGPoE: SMSE 0.00001475, MSLL 168.9830, NLPD 175.5819 - q   0.40
TEGPoE: SMSE 0.00001342, MSLL 30.2885, NLPD 36.8874 - q   0.60
TEGPoE: SMSE 0.00001114, MSLL -0.3546, NLPD 6.2443 - q   0.80
TEGPoE: SMSE 0.00000870, MSLL -7.6332, NLPD -1.0343 - q   1.00
TEGPoE: SMSE 0.00000836, MSLL -8.5321, NLPD -1.9333 - q   1.20
TEGPoE: SMSE 0.00001162, MSLL -5.6791, NLPD 0.9197 - q   1.40
TEGPoE: SMSE 0.00001748, MSLL 3.7104, NLPD 10.3093 - q   1.60
TEGPoE: SMSE 0.00002390, MSLL 30.1894, NLPD 36.7883 - q   1.80
TEGPoE: SMSE 0.00002956, MSLL 106.2221, NLPD 112.8210 - q   2.00
TEGPoE: SMSE 0.00003402, MSLL 333.3151, NLPD 339.9140 - q   2.20
TEGPoE: SMSE 0.00003733, MSLL 1038.2050, NLPD 1044.8039 - q   2.40
TEGPoE: SMSE 0.00003970, MSLL 3300.3193, NLPD 3306.9182 - q   2.60
TEGPoE: SMSE 0.00004136, MSLL 10765.3571, NLPD 10771.9560 - q   2.80
TEGPoE: SMSE 0.00004250, MSLL 35979.0122, NLPD 35985.6110 - q   

### 1.3.3 Find single q

In [11]:
criterion = 'TEGPoE';

In [12]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=1000, lr=1e-9, lambda=1e6, init_q=1.2);

Optimizing q
Start preprocessing...
Finish preprocessing...
Optimizing q: remain iter - 1000, q - 1.2000


In [15]:
q

q =  1.3273


In [13]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.3.4 Find multi q

In [17]:
printf('Optimizing q\r\n');
iqs = 1.2 + 0.01 * rand(opts.Ms,1)
for ldk=-1:3
    ld = 10^ldk
    qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-6, lambda=ld, init_q=iqs)
    [mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
endfor

Optimizing q
iqs =

   1.2053
   1.2068
   1.2068
   1.2077
   1.2078
   1.2048
   1.2037
   1.2000
   1.2059
   1.2039
   1.2062
   1.2082
   1.2078
   1.2006
   1.2065
   1.2039
   1.2062
   1.2092
   1.2018
   1.2022
   1.2028
   1.2010
   1.2092
   1.2080
   1.2032
   1.2054
   1.2029
   1.2068
   1.2031
   1.2036

ld =  0.10000
Start preprocessing...
k =  1
k =  2
k =  3
k =  4
k =  5
k =  6
k =  7
k =  8
k =  9
k =  10
k =  11
k =  12
k =  13
k =  14
k =  15
k =  16
k =  17
k =  18
k =  19
k =  20
k =  21
k =  22
k =  23
k =  24
k =  25
k =  26
k =  27
k =  28
k =  29
k =  30
Finish preprocessing...
qs =

   1.2046
   1.2071
   1.2069
   1.2074
   1.2087
   1.2059
   1.2031
   1.1999
   1.2061
   1.2047
   1.2061
   1.2081
   1.2085
   1.1998
   1.2076
   1.2033
   1.2055
   1.2100
   1.2020
   1.2016
   1.2028
   1.2019
   1.2093
   1.2088
   1.2029
   1.2046
   1.2031
   1.2077
   1.2034
   1.2036

TEGPoE: SMSE 0.00000842, MSLL -8.5209, NLPD -1.9220
ld =  1
Start preprocessing.